In [12]:
import LambdaRankHW
import query
import os

DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmpbrgqpkiq/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmpbrgqpkiq/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce 940MX (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 5110)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\tensor\signal\do

In [3]:
import numpy as np
def dcg(ranking_labels, k):
    gains = [(2 ** label - 1) / (np.log2(1 + (i + 1))) for i, label in enumerate(ranking_labels[:k])]  # Discounted gain at each rank
    return sum(gains)  # cumulated gains

def ndcg(ranking, k, best_ranking):
  return dcg(ranking, k)/dcg(best_ranking, k)

In [10]:
def dcg3(ranked_labels, k):
    gains = [(2**ranked_labels[i] -1 ) / (np.log2(2 + i)) for i in range(k)]
    return sum(gains)

In [4]:
def dcg2(g, rank=10):
    """
    Discounted Cumulative Gain implementation
    g - a list of ranking of relevance
    References:
        * Chapelle et al. Expected Reciprocal Rank for Graded Relevance. Formula (1).
    """
    # log(i + 2) instead of log(i + 1) because of our indexation that starts from 0
    return sum([(2 ** g[i] - 1) / np.log2(i + 2) for i in range(rank)])

In [6]:
%timeit dcg([5,5,5,5,5,5,5,5,5,5],k=10)

100000 loops, best of 3: 18.5 µs per loop
140.850339481


In [7]:
%timeit dcg2([5,5,5,5,5,5,5,5,5,5],rank=10)

The slowest run took 4.40 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 17.6 µs per loop


In [11]:
%timeit dcg3([5,5,5,5,5,5,5,5,5,5],k=10)

100000 loops, best of 3: 17.4 µs per loop


In [19]:
train_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/train.txt'), 64)
test_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/test.txt'), 64)
val_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/vali.txt'), 64)

loaded  0
loaded queries from file
loaded  0
loaded queries from file
loaded  0
loaded queries from file


In [20]:
print('training points', len(train_queries))
print('testing points', len(test_queries))
print('validation points', len(val_queries))

training points 90
testing points 30
validation points 30


In [32]:
num_docs = []
for query in train_queries:
    num_docs.append(query.get_document_count())

In [34]:
print(num_docs)

[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 197, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [47]:
train_qids = set(train_queries.keys())
test_qids = set(test_queries.keys())
val_qids = set(val_queries.keys())
all_qids = set.union(train_qids, test_qids, val_qids) 

In [76]:
from operator import itemgetter
print(sorted(all_qids))

['152', '155', '158', '161', '162', '163', '166', '167', '168', '172', '174', '175', '176', '181', '183', '184', '186', '187', '192', '195', '199', '200', '201', '205', '208', '222', '226', '230', '233', '235', '238', '244', '245', '252', '254', '255', '257', '258', '260', '261', '262', '263', '265', '266', '267', '268', '269', '271', '273', '279', '280', '283', '284', '287', '288', '290', '291', '292', '294', '297', '299', '300', '302', '305', '307', '308', '309', '311', '313', '316', '317', '318', '319', '321', '322', '326', '328', '330', '332', '333', '334', '336', '337', '338', '339', '341', '343', '344', '346', '349', '350', '355', '356', '357', '361', '363', '364', '365', '367', '370', '371', '372', '374', '375', '376', '377', '378', '380', '381', '384', '385', '386', '387', '388', '389', '390', '392', '393', '394', '395', '396', '397', '398', '401', '403', '406', '407', '412', '414', '415', '418', '419', '420', '421', '422', '425', '429', '432', '433', '434', '435', '436', '437'

In [69]:
query.get_docids()[0]
set([doc.get_id() for doc in query.get_docids()])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [70]:
doc_ids = set()

for x in train_queries:
    doc_ids = set.union(set([doc.get_id() for doc in x.get_docids()]), doc_ids)
    
for x in test_queries:
    doc_ids = set.union(set([doc.get_id() for doc in x.get_docids()]), doc_ids)

for x in val_queries:
    doc_ids = set.union(set([doc.get_id() for doc in x.get_docids()]), doc_ids)

In [71]:
print(len(doc_ids))

1000


In [82]:
S = np.zeros((1000,1000))
q = query
ds = [doc.get_id() for doc in q.get_docids()]
labels = q.get_labels()
for i in range(len(ds)):
    for j in range(len(ds)):
        if labels[i] > labels[j]:
            S[i,j] = 1
        elif labels[i] == labels[j]:
            S[i,j] = 0
        else:
            S[i,j] = -1

In [86]:
import pandas as pd
df = pd.DataFrame(S)

In [106]:
q_d = {}

with open(os.path.normpath('./HP2003/Fold1/train.txt'),'r') as f:
    for line in f:
        a = line.split()
        qid = a[1].replace('qid:','')
        
        if qid not in q_d:
            q_d[qid] = set()
        
        q_d[qid].add(a[-1]) # add doc id

In [112]:
print(q_d.keys())
q_d['201'] == q_d['200']
print(q_d['201'])
print(q_d['200'])

dict_keys(['201', '344', '200', '155', '292', '208', '230', '262', '328', '167', '263', '255', '283', '338', '341', '311', '192', '260', '313', '186', '266', '268', '158', '184', '166', '261', '174', '299', '319', '238', '337', '168', '287', '245', '257', '235', '308', '307', '302', '318', '332', '252', '181', '305', '258', '269', '284', '226', '222', '254', '199', '300', '175', '176', '162', '163', '322', '330', '187', '291', '346', '288', '271', '334', '161', '343', '183', '309', '172', '279', '316', '290', '152', '297', '336', '333', '205', '280', '317', '321', '195', '326', '349', '267', '294', '273', '265', '233', '244', '339'])
{'G24-85-0199913', 'G25-98-3043625', 'G17-36-3964843', 'G41-31-3560329', 'G44-41-2288667', 'G19-82-4049682', 'G06-24-2308338', 'G33-00-2275226', 'G36-16-2027955', 'G24-47-0137454', 'G02-13-0983306', 'G38-40-1394775', 'G18-45-3380895', 'G36-06-0187850', 'G08-86-0824906', 'G02-45-2948998', 'G12-03-2536719', 'G33-39-3274329', 'G10-30-2049467', 'G01-50-3924344